In [ ]:
%run ../talktools.py

[Run on Colab](https://colab.research.google.com/github/profjsb/python-seminar/blob/master/DataFiles_and_Notebooks/07_machine_learning_2/03_classification_and_CNNs_with_keras.ipynb)

# Classification with Keras

*AY 250 (UC Berkeley, 2012-2022)*

We've just seen regression problems with neural nets in `keras`. Let's now explore classification, on images. Let's introduce the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist#labels) dataset: 70k small (28$\times$28) images of 10 different types of clothing.

<img src="https://github.com/zalandoresearch/fashion-mnist/blob/master/doc/img/fashion-mnist-sprite.png?raw=true" width="80%">

Each training and test example is assigned to one of the following labels:

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

Tensorflow has a simple method to get this data locally

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
import datetime, os
import numpy as np
import warnings

from IPython.external import mathjax

import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Activation, BatchNormalization
)
from tensorflow.keras import backend as K
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
# Print keras version
print("Using tensorflow version:", tensorflow.keras.__version__)

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # scale the images to 0-1

In [ ]:
x_train.shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

ind = 21
plt.axis('off')
_ = plt.imshow(x_train[ind], cmap=plt.cm.gray_r, interpolation='nearest')

To learn a model to predict the class of a given image, we could treat this 28$\times$28 image as 1d input, like a stellar spectrum:

In [ ]:
_ = plt.plot(x_train[ind].reshape(-1))
plt.ylabel("normalized flux")
plt.xlabel("1D pixel index")

But this *clearly* jumbles the inherent spatial structure and local correlations found in natural images. Using just Dense layers in a NN we'd effectively be asking the network to learn these correlations.

## Convolutional Neural Nets (ConvNets)

NNs built for images (or more generally, inputs with spatial structure).

### Key Ideas: 
  - layers see only parts of each image (effectively all other weights are zero).
  - some layers do simple operations on previous layers to reduce dimensionality (e.g., take the largest value in a a 3x3 range)
  - "Every Layer has a simple API: It transforms an input 3D volume to an output 3D volume with some differentiable function that may or may not have parameters."
 
<img src="http://cs231n.github.io/assets/cnn/cnn.jpeg">

<img src="http://cs231n.github.io/assets/cnn/depthcol.jpeg">

"An example input volume in red (e.g. a 32x32x3 CIFAR-10 image), and an example volume of neurons in the first Convolutional layer. Each neuron in the convolutional layer is connected only to a local region in the input volume spatially, but to the full depth (i.e. all color channels). Note, there are multiple neurons (5 in this example) along the depth, all looking at the same region in the input - see discussion of depth columns in text below. "

cf. http://cs231n.github.io/convolutional-networks/

<img src="data/f2.png">
Source: http://www.nature.com/nature/journal/v521/n7553/fig_tab/nature14539_F2.html

### Filter banks

  http://setosa.io/ev/image-kernels/

### Pooling

<img src="http://cs231n.github.io/assets/cnn/pool.jpeg" width="40%">
<img src="http://cs231n.github.io/assets/cnn/maxpool.jpeg" width="40%">
Source: http://cs231n.github.io/convolutional-networks/

In [ ]:
from tensorflow.keras.utils import to_categorical

nb_classes = 10
batch_size = 128

# convert class vectors to binary class matrices
Y_train =  to_categorical(y_train, nb_classes)
Y_test =  to_categorical(y_test, nb_classes)

input_shape = x_train[0].shape  + (1,)
input_shape

In [ ]:
Y_train

We want our output predictions to look like a "probability" of belonging to one of the 10 classes. And, importantly, we'd like to make sure that the probability over all classes sums to unity. One way to do this is to scale the outputs of the last layer using a [`softmax`](https://en.wikipedia.org/wiki/Softmax_function):

$$
{\rm softmax}(\vec s) = \frac{e^{s_i}}{\sum_i e^{s_i}}
$$

So if the (unnormalized) prediction from am NN for an image is:

In [ ]:
s = np.random.normal(size=(10,))
s

Then the softmax scaling gives us:

In [ ]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))

print(softmax(s))
np.testing.assert_almost_equal(softmax(s).sum(), 1.0)

In [ ]:
from scipy.special import softmax as sp_softmax
sp_softmax(s)

We'll use the "categorical cross-entropy" loss:

<img src="https://gombru.github.io/assets/cross_entropy_loss/softmax_CE_pipeline.png">
Source: https://gombru.github.io/2018/05/23/cross_entropy_loss/

In [ ]:
# perfect match ... use a small ϵ to avoid taking log(0) since lim x log x -> 0 as x->0
print("loss with a perfect match:", -(Y_train[0] @ np.log(Y_train[0] + 1e-16)))
print("loss with a predicted match:", -(Y_train[0] @ np.log(softmax(s) + 1e-16)))

## Building a CNN

In [ ]:
# add with tf.device('/gpu:0'): if on GPU

#with tf.device('/gpu:0'):
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3,3), input_shape=input_shape))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(32, activation='relu'))


model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
run_time_string = datetime.datetime.utcnow().isoformat(timespec='minutes')
# define path to save model
model_path = f'nn_results/ay250_nn_{run_time_string}.h5'
print(f"Training ... {model_path}")

logdir = os.path.join("nn_results", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.75,
                              patience=2, min_lr=1e-6, verbose=1, cooldown=0)

csv_logger = tf.keras.callbacks.CSVLogger(f'nn_results/training_{run_time_string}.log')

earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, 
                                             patience=3, \
                                             verbose=1, mode='auto')

model_check = tf.keras.callbacks.ModelCheckpoint(model_path,
        monitor='val_accuracy', 
        save_best_only=True, 
        mode='max',
        verbose=1)

model.fit(x=x_train.reshape(-1,28,28,1), 
          y=Y_train, 
          epochs=20,
          batch_size=batch_size,
          validation_data=(x_test.reshape(-1,28,28,1), Y_test), 
          callbacks=[tensorboard_callback, reduce_lr, csv_logger, earlystop, model_check])

### Aside: Dropout 

You'll notice above that the `accuracy` is much higher than the `val_accuracy`. That is, we overfit on the training data. One way to help protect against this is to introduce `Dropout`

<img src="https://cdn-images-1.medium.com/max/1600/1*iWQzxhVlvadk6VAJjsgXgg.png">

Srivastava, Nitish, et al. ”Dropout: a simple way to prevent neural networks from
overfitting”, JMLR 2014

```python
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.1)) # 10% of dropping an output connection
```

### Aside: Visualization of the layers

From François Chollet (“DEEP LEARNING with Python”):

Intermediate activations are “useful for understanding how successive convnet layers transform their input, and for getting a first idea of the meaning of individual convnet filters.”

“The representations learned by convnets are highly amenable to visualization, in large part because they’re representations of visual concepts. Visualizing intermediate activations consists of displaying the feature maps that are output by various convolution and pooling layers in a network, given a certain input (the output of a layer is often called its activation, the output of the activation function). This gives a view into how an input is decomposed into the different filters learned by the network. Each channel encodes relatively independent features, so the proper way to visualize these feature maps is by independently plotting the contents of every channel as a 2D image.”

Following from https://github.com/gabrielpierobon/cnnshapes/blob/master/README.md

In [ ]:
layer_outputs = []
layer_names = []
for layer in model.layers:
    keep = True
    for t in ["dropout", "normalization", "flatten", "dense", "activation"]:
        if layer.name.find(t) != -1:
            keep = False
    if keep:
        layer_outputs.append(layer.output)
        layer_names.append(layer.name) 
        
activation_model = tensorflow.keras.models.Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(x_test[0].reshape(1,28,28,1)) # Returns a list of five Numpy arrays: one array per layer activation

In [ ]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

images_per_row = 15

for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps
    n_features = layer_activation.shape[-1]   # Number of features in the feature map
    size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
    n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): # Tiles each filter into a big horizontal grid
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, # Displays the grid
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

# Data Augmentation

Another way to avoid overfitting, aside from `Dropout`, is to increase the number of exmaples used to to train the model.  Data augmentation is a generic term for methods used to expand the effect training set size by generating more data from the original training set. In images, this is pretty natural: scale changes, rotations, flips, etc. should still give us the same label. This method has the benefit of usually increasing test-time accuracy.

<img src="https://cdn-images-1.medium.com/max/1200/1*C8hNiOqur4OJyEZmC7OnzQ.png">

In keras we can use the `ImageDataGenerator`. In Pytorch see https://pytorch-lightning.readthedocs.io/en/stable/extensions/datamodules.html

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# creating the data aumentation genreators for both the training images and the training label masks
data_gen_args = dict(featurewise_center=False,
                     featurewise_std_normalization=False,
                     rotation_range=15.,
                     rescale=1.0,
                     shear_range=0.1,
                     zoom_range=0.1,
                     horizontal_flip=True,
                     fill_mode = 'nearest')

image_datagen = ImageDataGenerator(**data_gen_args)

seed = 42
batch_size = 256


(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # scale the images to 0-1

# convert class vectors to binary class matrices
Y_train =  to_categorical(y_train, nb_classes)
Y_test =  to_categorical(y_test, nb_classes)

input_shape = x_train[0].shape  + (1,)

image_generator = image_datagen.flow(
    np.expand_dims(x_train, axis=3),  Y_train,
    shuffle = True,
    batch_size=batch_size,
    seed=seed)

In [ ]:
ImageDataGenerator?

In [ ]:
type(image_generator)

In [ ]:
rez = image_generator.next()

In [ ]:
rez[1]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(rez[0][4][:,:,0])

In [ ]:
# add with tf.device('/gpu:0'): if on GPU
#tf.reset_default_graph()
K.clear_session()

#with tf.device('/gpu:0'):
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.1))


model.add(Conv2D(64, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.1))


model.add(Conv2D(128, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Dropout(rate=0.1))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

# Dropout
model.add(Dropout(rate=0.2))

model.add(Dense(32, activation='relu'))


model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

run_time_string = datetime.datetime.utcnow().isoformat(timespec='minutes')
# define path to save model
model_path = f'nn_results/ay250_nn_{run_time_string}.h5'
print(f"Training ... {model_path}")

logdir = os.path.join("nn_results", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.75,
                              patience=2, min_lr=1e-6, verbose=1, cooldown=0)

csv_logger = tf.keras.callbacks.CSVLogger(f'nn_results/training_{run_time_string}.log')

earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, 
                                             patience=3, \
                                             verbose=1, mode='auto')

model_check = tf.keras.callbacks.ModelCheckpoint(model_path,
        monitor='val_accuracy', 
        save_best_only=True, 
        mode='max',
        verbose=1)

model.fit(image_generator,
          epochs=20,
          validation_data=(x_test.reshape(-1,28,28,1), Y_test), 
          callbacks=[tensorboard_callback, reduce_lr, csv_logger, earlystop, model_check])

In [ ]:
import pandas as pd

latest_log_file = !ls -t1 nn_results/training* | head -1
latest_model_file = !ls -t1 nn_results/ay250_*.h5 | head -1

hist_df = pd.read_csv(latest_log_file[0])
hist_df

In [ ]:
# reload the best model
from tensorflow.keras.models import load_model
import tensorflow as tf

model = load_model(latest_model_file[0])

In [ ]:
pred_y = model.predict(x_test.reshape(-1, 28, 28, 1))
pred_y

In [ ]:
pred_y.shape

In [ ]:
ind = 1010

print(pred_y[ind])

In [ ]:
pred_y[ind].sum()

In [ ]:
np.argmax(pred_y[ind])

In [ ]:
plt.axis('off')
_ = plt.imshow(x_test[ind], cmap=plt.cm.gray_r, interpolation='nearest')

In [ ]:
print(class_names[np.argmax(pred_y[ind])])